# Congestion Relief Market - Iterative Solution
#### Jupyter Notebook developed by Declan Heim for SOLA5050 Assignment T1 2022.




---
## Foreword
A brief context is provided here as abstracted from the accompanying report. This main script is used to preform the iteration solution for the CRM model in nempy.

### Research Question
How would the proposed Edify Energy’s Congestion Relief Market (CRM) affect the spot market profitability of Utility-PV generators in the NEM which are frequently involved in binding thermal constraints? 

### Case Study
The question is addressed through a defined case study. This case assesses historical data from the 15th of February 2021 in which considerable Utility-PV generation was curtailed in the Central-West Orana region of NSW, as reported by various sources (AEMO, 2021c; Simpson, 2021).

Specifically, the following parameters are defined in this case study:
- Simulation Period – historical dispatch interval data that is considered for simulation is confined to the 15th of February, 2021.
- Selected constraints – the considered thermal binding constraint equations for which the CRM is considered include:
    - Line 94T constraints: N>>N_NIL_94T, N>>N_NIL_94T_947.
    - Line 94K constraints: N>>N-PKWL_94K_1, N>>N-PKWL_94K_2, N>>N-PKWL_94K_3.
- Eligible Units – those units identifiable on the LHS of the selected constraints. These units are presented prior in Figure 2 and mapped to the respective constraints in the results section. Note that while Bodangora Wind Farm contributes to these constraints and is included in modelling, it has been omitted from the presented analysis given the focus here on utility-PV curtailment.

Note: 
1. Line 94K constraints have not been modelled but included for context with specific extracts in the appendix. 
2. A description of the constraint names is appended to the report as obtained from NEOmobile (Intelligent Energy Systems, 2022).

### Data
#### Input Data to the Model
This counter-factual data model requires extensive historical NEM data obtained from AEMO’s Market Management System (MMS) which is published to NEMWEB (AEMO, 2022b). Various data sources obtained from MMS are configured as inputs to the ‘nempy’ simulator. Given these inputs are configured by the simulation package, details of each AEMO dataset are omitted, however these are noted in an example simulation of nempy (Gorman, 2022) as well as accessible via the appended source-code. 
#### Output Data retrieved from the Model
By solving the economic dispatch model, nempy is able to provide market outcomes including prices, individual unit dispatch volumes, and information on binding constraints. Of the three key outcomes noted, market prices should remain unchanged before and after the CRM, unit dispatch volumes change in accordance with the CRM outcomes and finally non-zero marginal values  reflect where a specific constraint is bound.

### Method
#### High-level Overview
1. Using the case study parameters, an initial ‘base case’ energy dispatch simulation for a specific dispatch interval is performed in nempy. This yields the results of the historical market conditions.

    a. From the base case, it is verified that the ‘selected constraints’ are binding and units which are involved in the LHS of the constraint are identified. Only these units will be eligible to participate in a CRM associated with this constraint.
    
    
2.	A second iteration of the dispatch simulation is performed with the selected constraints entirely removed, establishing a ‘removed-constraint case’.

    a.	From this case, the impact that the selected constraint has on each unit is identified by the difference in dispatch between the ‘base case’ and ‘removed-constraint case’. This difference, typically an increase in dispatch volume in MWs, provides a maximum volume for congestion relief that can be bought by the respective unit for the associated CRM.
    
    
3.	A simple dispatch model of the associated CRM is created for the ‘selected constraints’ and solved. The structure would follow the example on page 6 of Edify’s submission (Stiel, 2021).

    a.	Only eligible participants bids are modelled – step I.a. above.
    
    b.	One congestion relief provider, a hypothetical utility-BESS, is created by adding this unit to the CRM.
    
    c.	The offer/bid price that each participant submits to CRM is predefined hypothetically.
    
    d.	A maximum cap is imposed on the amount of congestion relief bought, step II.a., and the price at which the congestion relief market can settle, being no greater than the energy price. 
    
    
4.	Having solved the CRM associated with the selected constraint, generic constraints are written into the nempy model to define the CRM preference for dispatch so that the congestion relief providers and buyers are dispatched in accordance with the CRM outputs.


5.	A third and final iteration of the energy dispatch simulation is performed given the considerations of step IV, and the addition of the hypothetical storage unit into the market.


6.	The results from the above procedure are collated from python, with the various output datasets visualized in Tableau. Repetitions of the above with variations to the CRM offer/bid prices can be considered as differing scenarios. 



---
<a id="Home"></a>
## Model Implementation

### Notebook Contents
- [Section A: Model Building](#A)
- [Section B: Iterative Solution](#B)
- [Section C: Aggregating Results](#C)


<a id="A"></a>

---

## Section A: Model Building
### A1. Import Python Packages + Configure Result File Structure

In [ ]:
%%capture
# Standard Packages
import pandas as pd
!pip install db-sqlite3
import sqlite3
import numpy as np
import os
from datetime import datetime, timedelta

# The forked nempy market dispatch engine is loaded with additional constraints features
!pip install --user --upgrade git+https://github.com/dec-heim/nempy_constraints_v1.git
from nempy import markets, time_sequential
from nempy.historical_inputs import loaders, mms_db, \
    xml_cache, units, demand, interconnectors, constraints

# Helper Functions
from crm_helper_functions import *
print()

In [ ]:
# File Structure is defined, customising the folder name per simulation run
folder_name = 'simulation_number_01'
fld_parent = os.path.join('results',folder_name)

fld_mv = os.path.join('results',folder_name,'marginal_values')
fld_ud = os.path.join('results',folder_name,'unit_dispatch')
fld_ep = os.path.join('results',folder_name,'energy_prices')
fld_erev = os.path.join('results',folder_name,'energy_revenue')
fld_err = os.path.join('results',folder_name,'error')
fld_dd = os.path.join('results',folder_name,'dispatch_diff')
fld_cp = os.path.join('results',folder_name,'crm_prices')
fld_cd = os.path.join('results',folder_name,'crm_dispatch')
fld_cr = os.path.join('results',folder_name,'crm_revenue')

if not os.path.exists(os.path.join('results',folder_name)):
    os.makedirs(fld_mv)
    os.makedirs(fld_ud)
    os.makedirs(fld_ep)
    os.makedirs(fld_erev)
    os.makedirs(fld_err)
    os.makedirs(fld_dd)
    os.makedirs(fld_cp)
    os.makedirs(fld_cd)
    os.makedirs(fld_cr)

---

### A2. Retrieve + Prepare Historical Data from AEMO

The data for this example has already been extracted and prepared for nempy locally. The data was found for the **15th February 2021** and includes a range of parameters, to name a few:
- unit details
- unit availability
- volume bids
- price bids
- unit ramp rate constraints
- interconnector models with loss equations
- regional demand constraints
- various other constraints such as FCAS, generic constraint sets

Further steps on how to download the data from AEMO MMS and configure this for nempy are provided in the [Detailed Recreation of Historical Dispatch Example](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) within nempy documentation.

In [ ]:
con = sqlite3.connect('feb_2021_mms.db')
mms_db_manager = mms_db.DBManager(connection=con)
xml_cache_manager = xml_cache.XMLCacheManager('feb_2021_cache')

raw_inputs_loader = loaders.RawInputsLoader(
    nemde_xml_cache_manager=xml_cache_manager,
    market_management_system_database=mms_db_manager)

---

### A3. Define Dispatch Intervals
For this specific example, we create dispatch intervals for **15th February 2021 from 6am to 7pm** to capture periods of utility-PV generation. This is formatted as a list for ease of later assessing multiple intervals.

Once the intervals are defined, the subsequent functions are called to load the respective data from our previously downloaded database.

In [ ]:
dispatch_intervals = create_dispatch_list('2021/02/15 06:00:00','2021/02/15 19:00:00')
di_df = pd.DataFrame(dispatch_intervals, columns=['di_list'])
di_df

---
### A4. Define the NEM Energy Market Model
The subsequent data loaded into the nempy market object follows the [Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) so we will omit this detail here for brevity and highlight only the generic constraints which are of interest in this analysis.

The <code>select_constraint</code> variable is used to define which constraints we analyse and hence which market the CRM is modelled for.
Equivalently, the <code>model_region</code> is defined

In [ ]:
select_constraint = ['N>>N-NIL_94T_947']
model_region = 'NSW1'

In [ ]:
def config_market(raw_inputs_loader, interval, add_relief_provider, relief):
     
    raw_inputs_loader.set_interval(interval)
    unit_inputs = units.UnitData(raw_inputs_loader)
    interconnector_inputs = interconnectors.InterconnectorData(raw_inputs_loader)
    constraint_inputs = constraints.ConstraintData(raw_inputs_loader)
    demand_inputs = demand.DemandData(raw_inputs_loader)

    # Define market, unit info
    unit_info = unit_inputs.get_unit_info()
    if add_relief_provider:
        unit_info = append_rp_unitinfo(unit_info, relief['new_storage'])
    market = markets.SpotMarket(market_regions=['QLD1', 'NSW1', 'VIC1',
                                                'SA1', 'TAS1'],
                                unit_info=unit_info)

    # Set volume, price bids
    volume_bids, price_bids = unit_inputs.get_processed_bids()
    if add_relief_provider:
        volume_bids = append_rp_volumebids(volume_bids, relief['new_storage'])
        price_bids = append_rp_pricebids(price_bids, relief['new_storage'])
    market.set_unit_volume_bids(volume_bids)
    market.set_unit_price_bids(price_bids)
    
    # Set bid in capacity limits
    unit_bid_limit = unit_inputs.get_unit_bid_availability()
    market.set_unit_bid_capacity_constraints(unit_bid_limit)
    cost = constraint_inputs.get_constraint_violation_prices()['unit_capacity']
    market.make_constraints_elastic('unit_bid_capacity', violation_cost=cost)

    # Set limits provided by the unconstrained intermittent generation
    # forecasts. Primarily for wind and solar.
    unit_uigf_limit = unit_inputs.get_unit_uigf_limits()
    market.set_unconstrained_intermitent_generation_forecast_constraint(
        unit_uigf_limit)
    cost = constraint_inputs.get_constraint_violation_prices()['uigf']
    market.make_constraints_elastic('uigf_capacity', violation_cost=cost)

    # Set unit ramp rates.
    ramp_rates = unit_inputs.get_ramp_rates_used_for_energy_dispatch()
    market.set_unit_ramp_up_constraints(
        ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_up_rate']])
    market.set_unit_ramp_down_constraints(
        ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_down_rate']])
    cost = constraint_inputs.get_constraint_violation_prices()['ramp_rate']
    market.make_constraints_elastic('ramp_up', violation_cost=cost)
    market.make_constraints_elastic('ramp_down', violation_cost=cost)

    # Set unit FCAS trapezium constraints.
    unit_inputs.add_fcas_trapezium_constraints()
    cost = constraint_inputs.get_constraint_violation_prices()['fcas_max_avail']
    fcas_availability = unit_inputs.get_fcas_max_availability()
    market.set_fcas_max_availability(fcas_availability)
    market.make_constraints_elastic('fcas_max_availability', cost)
    cost = constraint_inputs.get_constraint_violation_prices()['fcas_profile']
    regulation_trapeziums = unit_inputs.get_fcas_regulation_trapeziums()
    market.set_energy_and_regulation_capacity_constraints(regulation_trapeziums)
    market.make_constraints_elastic('energy_and_regulation_capacity', cost)
    scada_ramp_down_rates = unit_inputs.get_scada_ramp_down_rates_of_lower_reg_units()
    market.set_joint_ramping_constraints_lower_reg(scada_ramp_down_rates)
    market.make_constraints_elastic('joint_ramping_lower_reg', cost)
    scada_ramp_up_rates = unit_inputs.get_scada_ramp_up_rates_of_raise_reg_units()
    market.set_joint_ramping_constraints_raise_reg(scada_ramp_up_rates)
    market.make_constraints_elastic('joint_ramping_raise_reg', cost)
    contingency_trapeziums = unit_inputs.get_contingency_services()
    market.set_joint_capacity_constraints(contingency_trapeziums)
    market.make_constraints_elastic('joint_capacity', cost)

    # Set interconnector definitions, limits and loss models.
    interconnectors_definitions = \
        interconnector_inputs.get_interconnector_definitions()
    loss_functions, interpolation_break_points = \
        interconnector_inputs.get_interconnector_loss_model()
    market.set_interconnectors(interconnectors_definitions)
    market.set_interconnector_losses(loss_functions,
                                      interpolation_break_points)

    # Add FCAS market constraints.
    fcas_requirements = constraint_inputs.get_fcas_requirements()
    market.set_fcas_requirements_constraints(fcas_requirements)
    violation_costs = constraint_inputs.get_violation_costs()
    market.make_constraints_elastic('fcas', violation_cost=violation_costs) 

    # Add generic constraints, RHS parameters
    generic_rhs = constraint_inputs.get_rhs_and_type_excluding_regional_fcas_constraints()
    if add_relief_provider:
        generic_rhs = append_rbuy_rhs(generic_rhs, relief['crm_buyers'])
        violation_costs = append_rbuy_violationcosts(violation_costs, relief['crm_buyers'])
        generic_rhs = append_rbuy_rhs(generic_rhs, relief['crm_provider_rhs'])
        violation_costs = append_rbuy_violationcosts(violation_costs, relief['crm_provider_rhs'])
        
    market.set_generic_constraints(generic_rhs)
    market.make_constraints_elastic('generic', violation_cost=violation_costs)
    
    # Add generic constraints, LHS coefficients DUIDs
    unit_generic_lhs = constraint_inputs.get_unit_lhs()
    if add_relief_provider:
        unit_generic_lhs = append_rp_lhs(unit_generic_lhs, relief['new_storage'])
        unit_generic_lhs = append_rbuy_lhs(unit_generic_lhs, relief['crm_buyers'])
        unit_generic_lhs = append_rbuy_lhs(unit_generic_lhs, relief['crm_provider_rhs'])
    
    market.link_units_to_generic_constraints(unit_generic_lhs)
    
    # Add generic constraints, LHS coeffients ICs
    interconnector_generic_lhs = constraint_inputs.get_interconnector_lhs()
    market.link_interconnectors_to_generic_constraints(
        interconnector_generic_lhs)
    
    # Set the operational demand to be met by dispatch.
    regional_demand = demand_inputs.get_operational_demand()
    market.set_demand_constraints(regional_demand)
        
    return market

def economic_dispatch(raw_inputs_loader, interval, sim_mode=1, select_constraint=select_constraint, relief=None):
    """
    Defined modes to run economic dispatch:
    >> 1: Pure spot market economic dispatch [Default]
    >> 2: Spot market with constraint removed
    >> 3: Spot market with congestion relief provider
    """
    
    if sim_mode == 2:
        market = config_market(raw_inputs_loader,interval,add_relief_provider=False, relief=relief)
        # Check to remove constraint set
        if select_constraint is not None:
            for constraint in select_constraint:
                market.remove_generic_constraint_set(constraint)

        print("ED Complete: Select Constraint Removed!")
        market.dispatch()

    elif sim_mode == 3:
        market = config_market(raw_inputs_loader,interval,add_relief_provider=True, relief=relief)
        print("ED Complete: Relief Provider Added!")
        market.dispatch()
        
    else:
        # Default must run normal spot market
        market = config_market(raw_inputs_loader,interval,add_relief_provider=False, relief=relief)
        print("ED Complete: Default!")
        market.dispatch()
        
    # Retrieve & Save market data
    result_mv = market.get_constraint_marginal_values()
    result_mv.insert(0,'interval',interval)
    result_mv.insert(0,'sim_mode',sim_mode)
    
    result_units = market.get_unit_dispatch()
    result_units.insert(0,'interval',interval)
    result_units.insert(0,'sim_mode',sim_mode)

    result_prices = market.get_energy_prices()
    result_prices.insert(0,'interval',interval)
    result_prices.insert(0,'sim_mode',sim_mode)

    result_revenue = market_revenue(market._unit_info, result_prices, result_units)
    result_revenue.insert(0,'sim_mode',sim_mode)
        
    return market, {'marginal_values': result_mv, 'unit_dispatch': result_units, \
                    'energy_prices': result_prices, 'energy_revenue': result_revenue}

---
### A5. Define supporting functions needed for CRM implementation and analysis
Various additional functions are written to assist in modelling the CRM within nempy.

<code>get_dispatch_difference_byconst</code> and <code>get_dispatch_difference</code> are functions which compare two nempy market objects and return the difference in dispatch between iterations.

<code>get_eligible_units</code> is a function returning the units situated on the LHS of the defined selected constraints.

<code>create_run_crm_market</code> is the function constructing the CRM market which is solved considering the generators and loads defined within this function.

<code>format_crm_provider</code> is a function to format the crm information as output from the CRM model to be later interpreted by the economic dispatch functions in nempy.

In [ ]:
def get_dispatch_difference_byconst(output_default, output_const_removed, show_all=False):
    """
    Inputs: constraint_map, df of units for select_constraint,
    output_default: economic dispatch output market,
    output_const_removed: economic dispatch output market with constraint removed.
    """
    constraintmap = {}
    for constraint in select_constraint:
        
        # Get constraint mapping of DUIDs, coeff for select_constraint
        constraint_lhs = market.get_constraint_mapping(constraint)['units']    
        
        # Get original market output for these DUIDs
        select_units = output_default['unit_dispatch']
        select_units = select_units[(select_units['service'] == 'energy')\
                        & (select_units['unit'].isin(constraint_lhs['unit'].to_list()))]
        
        # Get modified market output for these DUIDs
        select_units2 = output_const_removed['unit_dispatch']
        select_units2 = select_units2[(select_units2['service'] == 'energy')\
                        & (select_units2['unit'].isin(constraint_lhs['unit'].to_list()))]
        
        # Calculate the difference in market output
        constraint_lhs = constraint_lhs.merge(right=select_units.loc[:,['unit','dispatch']], on='unit')
        constraint_lhs = constraint_lhs.merge(right=select_units2.loc[:,['unit','dispatch']], on='unit', suffixes=("","_r"))
        constraint_lhs['dispatch_diff'] = round(constraint_lhs['dispatch_r'] - constraint_lhs['dispatch'],2)
        
        if not show_all:
            constraintmap.update({constraint: constraint_lhs.loc[:,['set','unit','dispatch_diff']]})
        else:
            constraintmap.update({constraint: constraint_lhs})
    
    return constraintmap

def get_dispatch_difference(output_default, output_const_removed):
    """
    Inputs: constraint_map, df of units for select_constraint,
    output_default: economic dispatch output market,
    output_const_removed: economic dispatch output market with constraint removed.
    """
    constraintmap = {}
    
    # Get original market output for these DUIDs
    all_units = output_default['unit_dispatch']
    all_units = all_units[all_units['service'] == 'energy']
    
    # Get modified market output for these DUIDs
    all_units2 = output_const_removed['unit_dispatch']
    all_units2 = all_units2[all_units2['service'] == 'energy']
    
    # Calculate the difference in market output
    result = all_units.loc[:,['unit','dispatch']]
    result = result.merge(right=all_units2.loc[:,['unit','dispatch']], on='unit',suffixes=("","_r"))
    result['dispatch_diff'] = round(result['dispatch_r'] - result['dispatch'],2)
    
    return result

In [ ]:
# Function to retrieve eligible units to participate in CRM
def get_eligible_units(market, constraint_list):
    constraint_map, eligible_units_map = {},{}
    m_unitinfo = market._unit_info

    for constraint in constraint_list:
        const_units = market.get_constraint_mapping(constraint)['units']
        constraint_map.update({constraint: const_units})
        eligible_units_map.update({constraint: m_unitinfo[m_unitinfo['unit'].isin(const_units['unit'])]\
                                    .drop(['loss_factor','dispatch_type'],axis=1).reset_index(drop=True)})

    return eligible_units_map

In [ ]:
def create_run_crm_market(eligible_units_info, cap_price, cap_provision):

    new_crm = relief_market(eligible_units_info, cap_price)

    """
    For market to clear, the storage (provider) price must be below that of the buyer (any other unit)
    """
    new_crm.default_bids_offers(storage_mw=50.0, storage_offer=10.0)
    new_crm.bid_into_crm('BERYLSF1',price=20.0,volume=87.0) 
    new_crm.bid_into_crm('MANSLR1',price=1.0,volume= 46.0) 

    # Optional Setting
    #new_crm.cap_price(spotpricecap = cap_price) # set this to cap storage offer to energy price
    new_crm.cap_relief_provision(cap_provision) # cap dispatch volume by amount of possible relief
    
    return new_crm.dispatch(trim_price=False) # set trim price true to cap all at energy price

In [ ]:
def format_crm_provider(crm_provider):
    crm_provider.insert(1,'region', crm_results['prices']['region'][0])
    crm_provider.insert(2,'dispatch_type','load')
    crm_provider.insert(2,'loss_factor',1.0)
    crm_provider = crm_provider.rename(columns={'dispatch': 'relief_MW'})
    crm_provider.insert(2,'default_offer',15000.0)
    crm_provider.insert(2,'set',select_constraint[0])
    crm_provider.insert(2,'mirror_coeff','BERYLSF1')
    return crm_provider

---
[Return to Index](#Home)
<a id="B"></a>
## Section B: Model Simulation

### B1. Iteration Solution
For each dispatch interval the iteration process is considered, as commented throughout.

In [ ]:
outputs, outputs_mvs, outputs_prices, outputs_units, outputs_revenue = [],[],[],[], []

for di in di_df['di_list']:
    print(f"=== Solving Interval {di} ===")
    # Get base case economic dispatch results
    market, outputs = economic_dispatch(raw_inputs_loader, di, sim_mode=1)
        
    # Get economic dispatch results with constraint removed
    market_2, outputs_2 = economic_dispatch(raw_inputs_loader, di, sim_mode=2)
    
    # Find the dispatch difference for applicable units (eligble amount of congestion relief)
    dis_diff_12 = get_dispatch_difference(outputs, outputs_2)
    dis_diff_12.insert(0,'interval',di)
    dis_diff_12.insert(1,'id','1-2')
        
    # Find the units which can participate in the selected constraint for CRM
    eligible_units_info = get_eligible_units(market, select_constraint)
    
    # Construct Congestion Relief Market based on a hypothetical 'willingness to pay' for each unit in the constraint.
    # in here confine bid/offer by dispatch difference had the constraint not been binding.
    observed_prc = outputs['energy_prices']
    observed_prc = float(observed_prc[observed_prc['region'] == model_region]['price'])

    crm_results = create_run_crm_market(eligible_units_info[select_constraint[0]], \
                                        cap_price=observed_prc, cap_provision=dis_diff_12)
    
    crm_revenue = relief_revenue(crm_results['units'],crm_results['prices'], crm_results['dispatch'])
    
    # Formatting the changes to be applied from CRM dispatch to the NEM energy spot market.
    crm_data = crm_results['dispatch']
    
    crm_provider = crm_data[crm_data['unit'] == 'STORAGE']
    crm_provider = format_crm_provider(crm_provider)

    crm_buyer_param = crm_data[crm_data['unit'] != 'STORAGE']
    crm_buyer_param = format_crm_buyers(crm_buyer_param)
    
    crm_provider_rhs = crm_data[crm_data['unit'] == 'STORAGE']
    crm_provider_rhs = format_crm_buyers(crm_provider_rhs)

    relief_data = {'new_storage': crm_provider,
                  'crm_buyers': crm_buyer_param,
                  'crm_provider_rhs': crm_provider_rhs}
    
    # Simulating the final iteration
    # Economic Dispatch with CRM resolved.
    market_3, outputs_3 = economic_dispatch(raw_inputs_loader, di, sim_mode=3, relief=relief_data)
    final = outputs_3
    final.update({'error': validate_spot_with_crm_error(di, market, market_3)})
    
    # Differences in dispatch simulation runs
    dis_diff_13 = get_dispatch_difference(outputs, outputs_3)
    dis_diff_13.insert(0,'interval',di)
    dis_diff_13.insert(1,'id','1-3')
    final.update({'dispatch_diff': dis_diff_12})
    final['dispatch_diff'] = pd.concat([final['dispatch_diff'],dis_diff_13])
    
    # Collate CRM results
    crm_results['prices'].insert(0,'interval',di)
    crm_results['dispatch'].insert(0,'interval',di)
    crm_revenue.insert(0,'interval',di)
    final.update({'crm_prices': crm_results['prices']})
    final.update({'crm_dispatch': crm_results['dispatch']})
    final.update({'crm_revenue': crm_revenue})
    
    # Collate market results from all iterations
    final['marginal_values'] = pd.concat([final['marginal_values'],outputs['marginal_values'],outputs_2['marginal_values']])
    final['unit_dispatch'] = pd.concat([final['unit_dispatch'],outputs['unit_dispatch'],outputs_2['unit_dispatch']])
    final['energy_prices'] = pd.concat([final['energy_prices'],outputs['energy_prices'],outputs_2['energy_prices']])
    final['energy_revenue'] = pd.concat([final['energy_revenue'],outputs['energy_revenue'],outputs_2['energy_revenue']])
    
    # Convert the interval records from string to datetime objects
    for key in final:
        if 'interval' in final[key].columns:
            final[key]['interval'] = pd.to_datetime(final[key]['interval'], format='%Y/%m/%d %H:%M:%S')
            print(f"updated time format for {key}")
    
    # Format typeset to permit file saving
    if '/' in di:
        di = di.replace('/','-')
        di = di.replace(':','-')
    
    # Filter to save only specified units
    sou_list = ['BERYLSF1','BODWF1','GOONSF1','JEMALNG1','MANSLR1','MOLNGSF1','NEVERSF1','NYNGAN1','PARSF1','WELLSF1','STORAGE']
    final['unit_dispatch'] = final['unit_dispatch'][final['unit_dispatch']['unit'].isin(sou_list)]                                                    
    final['energy_revenue'] = final['energy_revenue'][final['energy_revenue']['unit'].isin(sou_list)]
    final['dispatch_diff'] = final['dispatch_diff'][final['dispatch_diff']['unit'].isin(sou_list)]
    
    # Download Results for each dataset to csv files
    final['marginal_values'].to_csv(os.path.join(fld_mv,f'mv_{di}.csv'),index=False)
    final['unit_dispatch'].to_csv(os.path.join(fld_ud,f'ud_{di}.csv'),index=False)
    final['energy_prices'].to_csv(os.path.join(fld_ep,f'ep_{di}.csv'),index=False)
    final['energy_revenue'].to_csv(os.path.join(fld_erev,f'erev_{di}.csv'),index=False)
    final['error'].to_csv(os.path.join(fld_err,f'err_{di}.csv'),index=False)
    final['dispatch_diff'].to_csv(os.path.join(fld_dd,f'dd_{di}.csv'),index=False)
    final['crm_prices'].to_csv(os.path.join(fld_cp,f'cp_{di}.csv'),index=False)
    final['crm_dispatch'].to_csv(os.path.join(fld_cd,f'cd_{di}.csv'),index=False)
    final['crm_revenue'].to_csv(os.path.join(fld_cr,f'cr_{di}.csv'),index=False)

---
[Return to Index](#Home)
<a id="C"></a>
## Section C: Aggregating Results

### C1. Reformat Results
Having run all dispatch intervals, the results files can be aggregated to a single summary file per dataset.

In [ ]:
import glob

for folder in os.listdir(fld_parent):
    glued_data = pd.DataFrame()
    for file in os.listdir(os.path.join(fld_parent,folder)):
        if file.endswith('.csv'):
            x = pd.read_csv(os.path.join(fld_parent,folder,file), low_memory=False, index_col=0)
            glued_data = pd.concat([glued_data,x],axis=0)
    glued_data.to_csv(os.path.join(fld_parent,f'agg_{folder}.csv'))
    
print("Finished data aggregation")

A further dataset is calculated by aggregating the energy and CRM revenue components as one net revenue field.

In [ ]:
# Net the revenue
crmrev = pd.read_csv(os.path.join(fld_parent,'agg_crm_revenue.csv'), low_memory=False)
energyrev = pd.read_csv(os.path.join(fld_parent,'agg_energy_revenue.csv'), low_memory=False)
energyrev = energyrev[energyrev['sim_mode'] == 3]
energyrev

netrev = pd.merge(left=energyrev, right=crmrev, left_on=['interval','unit'],right_on=['interval','unit'],suffixes=("_energy","_crm"))
netrev['revenue_total'] = netrev['revenue_energy'] + netrev['revenue_crm']
netrev.to_csv(os.path.join(fld_parent,f'agg_calc_net_revenue.csv'))